In [1]:
# Dependencies
import pandas as pd
import json
import requests
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import sqlite3

In [2]:
with open('Resources/ColoCitiesDataClean.csv') as f:
    print(f)

<_io.TextIOWrapper name='Resources/ColoCitiesDataClean.csv' mode='r' encoding='cp1252'>


In [3]:
latlon_df = pd.read_csv("Resources/ColoCitiesDataClean.csv", encoding='cp1252')
latlon_df.head()

,Place Name,lat,lon,Population
0,Aguilar,37.403611,-104.655000,480
1,Akron,40.163333,-103.220278,"1,723"
2,Alamosa,37.475278,-105.876944,"9,591"
3,Alma,39.285833,-106.066944,313
4,Antonito,37.076111,-106.010278,747


In [4]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/air_pollution?"
api_key= "90d059c74bd13bcc4087b66e982c1bf1"
lat=37.40361111
lon=-104.655

#Build URL
query_url = f"{url}lat={lat}&lon={lon}&appid={api_key}"

#Query API
air_response = requests.get(query_url).json()

#Test API response
print(air_response)

{'coord': {'lon': -104.655, 'lat': 37.4036}, 'list': [{'main': {'aqi': 2}, 'components': {'co': 233.65, 'no': 0.01, 'no2': 0.5, 'o3': 95.84, 'so2': 0.62, 'pm2_5': 10.02, 'pm10': 11.37, 'nh3': 0.69}, 'dt': 1626559200}]}


In [6]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/air_pollution?"
api_key= "90d059c74bd13bcc4087b66e982c1bf1"

# Counter to sort through response
number = 0
group = 1

# Create DataFrame
columns= ["City","Date", "lat", "lon", "AQI", "CO", "NO", "NO2", "O3", "SO2", "PM 2.5", "PM 10", "NH3"]
air_df = pd.DataFrame(columns=columns)

for index, row in latlon_df.iterrows():
    
    lat = row["lat"]
    lon = row["lon"]
    #print(lat)
    #print(lon)
    query_url = f"{url}lat={lat}&lon={lon}&appid={api_key}"
    air_response = requests.get(query_url).json()
 
    try:
        print(f"Processing Record {number}  of Group {group} | {lat}{lon}.")
        
        print (query_url) 
    
        air_api_values= {
            "City" : row["Place Name"],
            "Population": row["Population"],
            "Date" : air_response["list"][0]["dt"],  
             "lon" : air_response["coord"]["lon"],
             "lat" : air_response["coord"]["lat"],
             "AQI" : air_response["list"][0]["main"]["aqi"],
             "CO" : air_response["list"][0]["components"]["co"],  
             "NO" : air_response["list"][0]["components"]["no"],
             "NO2" : air_response["list"][0]["components"]["no2"],  
             "O3" : air_response["list"][0]["components"]["o3"],
             "SO2" : air_response["list"][0]["components"]["so2"],
             "PM 2.5" : air_response["list"][0]["components"]["pm2_5"],
             "PM 10" : air_response["list"][0]["components"]["pm10"],
             "NH3" : air_response["list"][0]["components"]["nh3"]
           }
        air_df= air_df.append(air_api_values, ignore_index=True)
        
    except:
        print(f"not working")
        pass
        
    
    # To save money on API calls
    number = number + 1
    if number == 50:
        number= 0
        group = group + 1


Processing Record 0  of Group 1 | 37.40361111-104.655.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.40361111&lon=-104.655&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 1  of Group 1 | 40.16333333-103.22027779999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.16333333&lon=-103.22027779999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 2  of Group 1 | 37.47527778-105.87694440000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.47527778&lon=-105.87694440000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 3  of Group 1 | 39.28583333-106.06694440000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.28583333&lon=-106.06694440000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 4  of Group 1 | 37.07611111-106.0102778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.07611111&lon=-106.0102778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 5  of Group 1 | 39.28444444-

Processing Record 44  of Group 1 | 38.81916667-102.35194440000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.81916667&lon=-102.35194440000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 45  of Group 1 | 38.36222222-105.1422222.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.36222222&lon=-105.1422222&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 46  of Group 1 | 37.14416667-104.6216667.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.14416667&lon=-104.6216667&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 47  of Group 1 | 39.24-107.96361110000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.24&lon=-107.96361110000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 48  of Group 1 | 38.8675-104.7605556.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.8675&lon=-104.7605556&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 49  of Group 1 | 39.59944444-105.03527779999999.


Processing Record 38  of Group 2 | 40.5475-105.06555559999998.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.5475&lon=-105.06555559999998&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 39  of Group 2 | 40.08277778-104.8069444.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.08277778&lon=-104.8069444&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 40  of Group 2 | 40.25361111-103.7902778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.25361111&lon=-103.7902778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 41  of Group 2 | 38.68833333-104.6827778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.68833333&lon=-104.6827778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 42  of Group 2 | 38.13055556-104.02638890000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.13055556&lon=-104.02638890000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 43  of Group 2 | 39.58805556-104.7858

Processing Record 32  of Group 3 | 40.985-102.2627778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.985&lon=-102.2627778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 33  of Group 3 | 40.13277778-104.53805559999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.13277778&lon=-104.53805559999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 34  of Group 3 | 40.39083333-104.5794444.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.39083333&lon=-104.5794444&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 35  of Group 3 | 37.24694444-103.3533333.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.24694444&lon=-103.3533333&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 36  of Group 3 | 39.34361111-104.45777779999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.34361111&lon=-104.45777779999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 37  of Group 3 | 38.76277778-102.795277

Processing Record 26  of Group 4 | 38.90833333-106.96055559999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.90833333&lon=-106.96055559999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 27  of Group 4 | 39.77472222-105.05666670000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.77472222&lon=-105.05666670000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 28  of Group 4 | 37.9325-107.8577778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.9325&lon=-107.8577778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 29  of Group 4 | 38.21833333-108.56666670000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.21833333&lon=-108.56666670000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 30  of Group 4 | 39.96333333-105.5072222.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.96333333&lon=-105.5072222&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 31  of Group 4 | 39.577

Processing Record 20  of Group 5 | 38.08638889-106.1411111.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.08638889&lon=-106.1411111&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 21  of Group 5 | 38.53-105.9977778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.53&lon=-105.9977778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 22  of Group 5 | 37.2025-105.4225.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.2025&lon=-105.4225&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 23  of Group 5 | 37.2575-105.90083329999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.2575&lon=-105.90083329999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 24  of Group 5 | 37.99472222-108.0022222.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.99472222&lon=-108.0022222&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 25  of Group 5 | 40.93555556-102.5255556.
http://api.openweathermap.org/data/

Processing Record 14  of Group 6 | 38.15527778-102.7191667.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.15527778&lon=-102.7191667&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 15  of Group 6 | 38.38388889-105.1711111.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.38388889&lon=-105.1711111&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 16  of Group 6 | 40.46888889-104.92.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.46888889&lon=-104.92&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 17  of Group 6 | 39.87861111-105.78277779999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.87861111&lon=-105.78277779999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 18  of Group 6 | 38.99861111-105.05944440000002.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.99861111&lon=-105.05944440000002&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 19  of Group 6 | 40.08-102.2283333.
htt

In [7]:
print(air_df)

              City        Date      lat       lon AQI      CO    NO   NO2  \
0          Aguilar  1626559200  37.4036 -104.6550   2  233.65  0.01  0.50   
1            Akron  1626559200  40.1633 -103.2203   2  230.31  0.03  0.61   
2          Alamosa  1626559200  37.4753 -105.8769   1  205.28  0.02  0.49   
3             Alma  1626559200  39.2858 -106.0669   2  186.92  0.03  0.50   
4         Antonito  1626559200  37.0761 -106.0103   1  195.27  0.01  0.17   
..             ...         ...      ...       ...  ..     ...   ...   ...   
267    Winter Park  1626559200  39.8786 -105.7828   2  185.25  0.01  0.11   
268  Woodland Park  1626559200  38.9986 -105.0594   2  198.60  0.01  0.21   
269           Wray  1626559200  40.0800 -102.2283   2  236.99  0.02  0.31   
270          Yampa  1626559200  40.1531 -106.9086   1  186.92  0.03  0.34   
271           Yuma  1626559200  40.1236 -102.7161   2  208.62  0.04  0.44   

         O3   SO2  PM 2.5  PM 10   NH3 Population  
0     95.84  0.62   10.

In [8]:
# obtain datetime series:
datetimes = pd.to_datetime(air_df['Date'], unit='s')

# assign your new columns
air_df['Year'] = datetimes.dt.year
air_df['Month'] = datetimes.dt.month
air_df['Day'] = datetimes.dt.day
air_df['Hour'] = datetimes.dt.hour
air_df['Minute'] = datetimes.dt.minute
air_df['Second'] = datetimes.dt.second

In [9]:
#Clean city names
air_df["City"] = air_df['City'].str.split('[').str[0]
air_df.head(50)

,City,Date,lat,lon,AQI,CO,NO,NO2,O3,SO2,PM 2.5,PM 10,NH3,Population,Year,Month,Day,Hour,Minute,Second
0,Aguilar,1626559200,37.4036,-104.6550,2,233.65,0.01,0.50,95.84,0.62,10.02,11.37,0.69,480,2021,7,17,22,0,0
1,Akron,1626559200,40.1633,-103.2203,2,230.31,0.03,0.61,113.01,0.39,14.26,16.42,1.01,"1,723",2021,7,17,22,0,0
2,Alamosa,1626559200,37.4753,-105.8769,1,205.28,0.02,0.49,75.82,0.22,8.11,9.40,0.68,"9,591",2021,7,17,22,0,0
3,Alma,1626559200,39.2858,-106.0669,2,186.92,0.03,0.50,65.80,0.22,10.19,11.29,0.46,313,2021,7,17,22,0,0
4,Antonito,1626559200,37.0761,-106.0103,1,195.27,0.01,0.17,71.53,0.47,7.86,8.82,0.43,747,2021,7,17,22,0,0
5,Arriba,1626559200,39.2844,-103.2742,2,211.95,0.03,0.36,94.41,0.38,10.20,11.94,1.01,207,2021,7,17,22,0,0
6,Arvada,1626559200,39.8319,-105.1511,2,206.95,0.04,0.66,101.57,1.01,9.30,10.79,0.51,"121,272",2021,7,17,22,0,0
7,Aspen,1626559200,39.1950,-106.8367,2,181.91,0.01,0.25,65.80,0.21,10.17,12.24,0.55,"7,401",2021,7,17,22,0,0
8,Ault,1626559200,40.5892,-104.7392,3,230.31,0.06,0.80,134.47,0.87,17.66,20.30,1.06,"1,871",2021,7,17,22,0,0
9,Aurora,1626559200,39.7083,-104.7239,3,240.33,0.15,2.16,150.20,1.45,16.83,18.82,0.80,"379,289",2021,7,17,22,0,0


In [11]:
air_df.to_csv('Resources/air_api_data.csv')
##Delete when merged
air_df.to_csv('Resources/data/air_api_data.csv')